**CODE EXECUTOR:** Dinh-Tu Vo

**Email:** 18521589@gm.uit.edu.vn

# CenterTrack - Tracking Objects as Points

**Original code :** [CenterTrack](https://github.com/xingyizhou/CenterTrack) by [Xingyi Zhou](https://github.com/xingyizhou)

**[Tracking Objects as Points](http://arxiv.org/abs/2004.01177)**,

Xingyi Zhou, Vladlen Koltun, Philipp Krähenbühl,

*arXiv technical report [(arXiv 2004.01177)](http://arxiv.org/abs/2004.01177)*

**Citation**
```
@article{zhou2020tracking,
  title={Tracking Objects as Points},
  author={Zhou, Xingyi and Koltun, Vladlen and Kr{\"a}henb{\"u}hl, Philipp},
  journal={ECCV},
  year={2020}
}
```

In [1]:
! nvidia-smi
!/usr/local/cuda/bin/nvcc --version

Thu Jul 15 16:22:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Installation
1. Install PyTorch:

In [2]:
import torch
print(torch.__version__)

1.9.0+cu102


In [3]:
! pip install torch==1.4.0 torchvision==0.5.0

     |████████████████████████████████| 753.4MB 22kB/s 
     |████████████████████████████████| 4.0MB 40.2MB/s 
ERROR: torchtext 0.10.0 has requirement torch==1.9.0, but you'll have torch 1.4.0 which is incompatible.
  Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102


2. Install [COCOAPI](https://github.com/cocodataset/cocoapi):

In [4]:
! pip install cython; pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-s2fiwb8x
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-s2fiwb8x
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=263904 sha256=26dd2d08e55e95bf4a62bb1de9759b4d5256c58aa9ce2f638eadc2f2eccc56ad
  Stored in directory: /tmp/pip-ephem-wheel-cache-9l4to_hh/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0.2
    Uninstalling pycocotools-2.0.2:
      Successfully uninstalled pycocotools-2.0.2


3. Clone this repo:

In [5]:
CenterTrack_ROOT="CenterTrack"
! git clone --recursive https://github.com/xingyizhou/CenterTrack $CenterTrack_ROOT
%cd CenterTrack

Cloning into 'CenterTrack'...
remote: Enumerating objects: 285, done.
remote: Total 285 (delta 0), reused 0 (delta 0), pack-reused 285
Receiving objects: 100% (285/285), 15.01 MiB | 28.72 MiB/s, done.
Resolving deltas: 100% (113/113), done.
/content/CenterTrack


You can manually install the [submodules](https://github.com/xingyizhou/CenterTrack/blob/master/.gitmodules) if you forget --recursive.

4. Install the requirements

In [6]:
! pip install -r requirements.txt
! apt-get install libx264-dev

     |████████████████████████████████| 286kB 10.8MB/s 
     |████████████████████████████████| 153kB 26.7MB/s 
     |████████████████████████████████| 7.1MB 35.6MB/s 
     |████████████████████████████████| 92kB 11.9MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 81kB 11.8MB/s 
     |████████████████████████████████| 102kB 13.1MB/s 
     |████████████████████████████████| 71kB 10.0MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
  Created wheel for progress: filename=progress-1.5-cp37-none-any.whl size=8088 sha256=8d6174bd9740508615a3c3e10f56bf360ad31f264d8d1298ffeb87707ab1a26d
  Stored in directory: /root/.cache/pip/wheels/6c/c8/80/32a294e3041f006c661838c05a411c7b7ffc60ff939d14e116
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=263978 sha256=0160e55fa010e852a4f808ffeee588e369fc974e458a8e886b48f6575c91dd7c
  Stored in directory: /root/.cache/pip/wheels/68/a5/e7/56401832f23d0b2d

In [7]:
%ls src/lib/model/networks/

backbones/     dla.py    generic_network.py  resdcn.py
base_model.py  dlav0.py  necks/              resnet.py


In [ ]:
#%rm -r src/lib/model/networks/DCNv2

5. Compile deformable convolutional (from DCNv2).

In [8]:
# clone if it is not automatically downloaded by `--recursive`.
%%shell

cd src/lib/model/networks/
git clone https://github.com/CharlesShang/DCNv2/  
cd DCNv2/
./make.sh

Cloning into 'DCNv2'...
remote: Enumerating objects: 214, done.
remote: Total 214 (delta 0), reused 0 (delta 0), pack-reused 214
Receiving objects: 100% (214/214), 1.42 MiB | 24.17 MiB/s, done.
Resolving deltas: 100% (121/121), done.
running build
running build_ext
building '_ext' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/content
creating build/temp.linux-x86_64-3.7/content/CenterTrack
creating build/temp.linux-x86_64-3.7/content/CenterTrack/src
creating build/temp.linux-x86_64-3.7/content/CenterTrack/src/lib
creating build/temp.linux-x86_64-3.7/content/CenterTrack/src/lib/model
creating build/temp.linux-x86_64-3.7/content/CenterTrack/src/lib/model/networks
creating build/temp.linux-x86_64-3.7/content/CenterTrack/src/lib/model/networks/DCNv2
creating build/temp.linux-x86_64-3.7/content/CenterTrack/src/lib/model/networks/DCNv2/src
creating build/temp.linux-x86_64-3.7/content/CenterTrack/src/lib/model/networks/DCNv2/src/cpu
creatin

## Download pertained models

Download pretrained models for [monocular 3D tracking](https://drive.google.com/open?id=1e8zR1m1QMJne-Tjp-2iY_o81hn2CiQRt), [80-category tracking](https://drive.google.com/open?id=1tJCEJmdtYIh8VuN8CClGNws3YO7QGd40), or [pose tracking](https://drive.google.com/open?id=1tJCEJmdtYIh8VuN8CClGNws3YO7QGd40) and move them to 
`$CenterTrack_ROOT/models/`. 

  More models can be found in [Model zoo](https://github.com/xingyizhou/CenterTrack/blob/master/readme/MODEL_ZOO.md).

In [ ]:
%mkdir models
#Download pretrained model of authur
#! gdown --id 1e8zR1m1QMJne-Tjp-2iY_o81hn2CiQRt -O models/nuScenes_3Dtracking.pth
! gdown --id 1tJCEJmdtYIh8VuN8CClGNws3YO7QGd40 -O models/coco_tracking.pth
#! gdown --id 1H0YvFYCOIZ06EzAkC2NxECNQGXxK27hH -O models/coco_pose_tracking.pth
! gdown --id 1sf1bWJ1LutwQ_wp176nd2Y3HII9WeFf0 -O models/mot17_half.pth
! gdown --id 1h_8Ts11rf0GQ4_n6FgmCeBuFcWrRjJfa -O models/mot17_fulltrain.pth

#Download our pretrained model
! gdown --id 1EFuV3FvU0CNMKY3wUCh9wr8UAp1o2Mjx -O models/model_last.pth

Downloading...
From: https://drive.google.com/uc?id=1tJCEJmdtYIh8VuN8CClGNws3YO7QGd40
To: /content/CenterTrack/src/lib/models/coco_tracking.pth
79.5MB [00:00, 219MB/s]
Downloading...
From: https://drive.google.com/uc?id=1sf1bWJ1LutwQ_wp176nd2Y3HII9WeFf0
To: /content/CenterTrack/src/lib/models/mot17_half.pth
80.0MB [00:00, 142MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1h_8Ts11rf0GQ4_n6FgmCeBuFcWrRjJfa
To: /content/CenterTrack/src/lib/models/mot17_fulltrain.pth
80.0MB [00:00, 92.4MB/s]


## Use CenterTrack

**Using dataset MOT17**

In [ ]:
%%shell
cd /content/CenterTrack/src/tools
sh get_mot_17.sh

In [ ]:
%cd /content/CenterTrack

/content/CenterTrack


In [ ]:
%mv /content/CenterTrack/data /content/CenterTrack/src/lib

In [9]:
from google.colab import drive
drive.mount('/content/dinhtu')

Mounted at /content/dinhtu


In [14]:
%cd /content/CenterTrack/src

/content/CenterTrack/src


## Demo on model pretrained of author 

**Note:**
1. replace "cv2.imshow" in function show_all_image in "debugger.py(/src/lib/utils/)" with "pass"
2. comment out  "cv2.imshow" at line 80 in "demo.py".

Video clip from the [nuScenes dataset](https://www.nuscenes.org/?externalData=all&mapData=all&modalities=Any) in `videos/nuscenes_mini.mp4`. To test monocular 3D tracking on this video, run

In [ ]:
! python demo.py tracking,ddd --load_model ../models/nuScenes_3Dtracking.pth --dataset nuscenes --pre_hm --track_thresh 0.1 --demo ../videos/nuscenes_mini.mp4 --test_focal_length 633 --save_video --video_h 480 --video_w 720 --debug 2
#! python demo.py tracking,ddd --load_model ../models/nuScenes_3Dtracking.pth --dataset nuscenes --pre_hm --track_thresh 0.1 --demo road_traffic.avi --save_video

You will need to specify test_focal_length for monocular 3D tracking demo to convert the image coordinate system back to 3D. The value 633 is half of a typical focal length (~1266) in nuScenes dataset in input resolution 1600x900. The mini demo video is in an input resolution of 800x448, so we need to use a half focal length. You don't need to set the test_focal_length when testing on the original nuScenes data.

Similarly, for 80-category tracking on images/ video, run:

In [ ]:
! python demo.py tracking --load_model ../models/coco_tracking.pth --demo content/CenterTrack/car.mp4

In [ ]:
! python demo.py tracking --load_model ../models/coco_tracking.pth --demo content/CenterTrack/car.mp4

If you want to test with person tracking models, you need to add --num_class 1:

In [ ]:
! python demo.py tracking --load_model ../models/mot17_half.pth --num_class 1 --demo pedestrian.avi --save_video
#! python demo.py tracking --load_model ../models/mot17_fulltrain.pth --num_class 1 --demo pedestrian.avi --save_video --debug 2

You can add --debug 2 to visualize the heatmap and offset predictions.

In [ ]:
! python demo.py tracking --load_model ../models/mot17_half.pth --num_class 1 --debug 2 --demo video.mp4

For monocular 3D tracking, run

In [ ]:
! python demo.py tracking,ddd --demo ../road_traffic.mp4 --load_model ../models/coco_tracking.pth --save_video

Similarly, for pose tracking, run:

In [ ]:
! python demo.py tracking,multi_pose --load_model ../models/coco_pose.pth --demo video.mp4

# Training Centertrack on our custom dataset


In [ ]:
%%time
! python main.py tracking --exp_id vehiclesMOT_fulltrain_mymodel --dataset custom --custom_dataset_ann_path /content/dinhtu/MyDrive/vehicles_detection/Tu_workspace/train_yolo.json --custom_dataset_img_path /content/dinhtu/MyDrive/vehicles_detection/Tu_workspace/train_yolo --input_h 416 --input_w 416 --num_classes 5 --num_epochs 15 --pre_hm --ltrb_amodal --same_aug --hm_disturb 0.05 --lost_disturb 0.4 --fp_disturb 0.1 --gpus 0 --use_kpt_center --eval_val --same_aug_pre --save_results  --save_all 

Running tracking
Using tracking threshold for out threshold! 0.3
Fix size testing.
training chunk_sizes: [32]
input h w: 416 416
heads {'hm': 5, 'reg': 2, 'wh': 2, 'tracking': 2, 'ltrb_amodal': 4}
weights {'hm': 1, 'reg': 1, 'wh': 0.1, 'tracking': 1, 'ltrb_amodal': 0.1}
head conv {'hm': [256], 'reg': [256], 'wh': [256], 'tracking': [256], 'ltrb_amodal': [256]}
Namespace(K=100, add_05=False, amodel_offset_weight=1, arch='dla_34', aug_rot=0, backbone='dla34', batch_size=32, chunk_sizes=[32], custom_dataset_ann_path='/content/dinhtu/MyDrive/vehicles_detection/Tu_workspace/train_yolo.json', custom_dataset_img_path='/content/dinhtu/MyDrive/vehicles_detection/Tu_workspace/train_yolo', data_dir='/content/CenterTrack/src/lib/../../data', dataset='custom', dataset_version='', debug=0, debug_dir='/content/CenterTrack/src/lib/../../exp/tracking/vehiclesMOT_fulltrain_mymodel/debug', debugger_theme='white', demo='', dense_reg=1, dep_weight=1, depth_scale=1, dim_weight=1, dla_node='dcn', down_ratio=

In [ ]:
%%time
! python main.py tracking --exp_id vehiclesMOT_fulltrain_mymodel --dataset custom --custom_dataset_ann_path /content/dinhtu/MyDrive/vehicles_detection/Tu_workspace/train.json --custom_dataset_img_path /content/dinhtu/MyDrive/vehicles_detection/Tu_workspace/train_yolo/images --input_h 512 --input_w 512 --num_classes 5 --num_epochs 15 --pre_hm --ltrb_amodal --same_aug --hm_disturb 0.05 --lost_disturb 0.4 --fp_disturb 0.1 --gpus 0 --save_results --eval_val --same_aug_pre --save_all --use_kpt_center

## Benchmark Evaluation and Testing
After installation, follow the instructions in [DATA.md](https://github.com/xingyizhou/CenterTrack/blob/master/readme/DATA.md) to setup the datasets. Then check [GETTING_STARTED.md](https://github.com/xingyizhou/CenterTrack/blob/master/readme/GETTING_STARTED.md) to reproduce the results in the paper. We provide scripts for all the experiments in the [experiments folder](https://github.com/xingyizhou/CenterTrack/blob/master/experiments).



**MOT17**

To test the tracking performance on MOT17 with our pretrained model, run

In [ ]:
%%time
! python test.py tracking --exp_id mot17_test --dataset mot --dataset_version 17test --pre_hm --ltrb_amodal --track_thresh 0.4 --pre_thresh 0.5 --load_model /content/CenterTrack/models/model_last.pth --save_results

`--pre_hm` is to enable the input heatmap. `--ltrb_amodal` is to use the left, top, right, bottom bounding box representation to enable detecting out-of-image bounding box (We observed this is important for MOT datasets). And `--track_thresh` and `--pre_thresh` are the score threshold for predicting a bounding box ($\theta$ in the paper) and feeding the heatmap to the next frame ($\tau$ in the paper), respectively.

To test with public detection, run

In [ ]:
! python test.py tracking --exp_id mot17_half_public --dataset mot --dataset_version 17halfval --pre_hm --ltrb_amodal --track_thresh 0.4 --pre_thresh 0.5 --load_model /content/CenterTrack/models/model_last.pth --public_det -- load_results ../data/mot17/results/val_half_det.json

**CUSTOM DATA**

To test with our custom dataset, run

In [17]:
%%time
! python test.py tracking --exp_id vehiclesMOT_test_mymodel --dataset custom --custom_dataset_ann_path /content/dinhtu/MyDrive/vehicles_detection/Tu_workspace/test_yolo.json --custom_dataset_img_path /content/dinhtu/MyDrive/vehicles_detection/Tu_workspace/test_yolo --input_h 512 --input_w 512 --num_classes 5 --trainval --pre_hm --ltrb_amodal --track_thresh 0.4 --pre_thresh 0.5 --load_model /content/CenterTrack/models/model_last.pth --save_results --eval_val --same_aug_pre --save_all

Streaming output truncated to the last 5000 lines.
vehiclesMOT_test_mymodel |                                | [38/1288]|Tot: 0:00:14 |ETA: 0:08:42 |tot 0.056s (0.057s) |load 0.000s (0.000s) |pre 0.001s (0.001s) |net 0.052s (0.053s) |dec 0.002s (0.002s) |post 0.000s (0.000s) |merge 0.000s (0.000s) |track 0.000s (0.000s) 
No pre_dets for 40 . Use empty initialization.
Start tracking video 40
Initialize tracking!
vehiclesMOT_test_mymodel |                                | [39/1288]|Tot: 0:00:14 |ETA: 0:08:42 |tot 0.057s (0.057s) |load 0.000s (0.000s) |pre 0.001s (0.001s) |net 0.052s (0.053s) |dec 0.002s (0.002s) |post 0.001s (0.000s) |merge 0.000s (0.000s) |track 0.000s (0.000s) 
No pre_dets for 41 . Use empty initialization.
Start tracking video 41
Initialize tracking!
vehiclesMOT_test_mymodel |#                               | [40/1288]|Tot: 0:00:15 |ETA: 0:08:41 |tot 0.057s (0.057s) |load 0.000s (0.000s) |pre 0.001s (0.001s) |net 0.053s (0.053s) |dec 0.003s (0.002s) |post 0.000s (0.00

#EVALUTION TRACKING

In [ ]:
! git clone --recursive https://github.com/cheind/py-motmetrics
%cd py-motmetrics

Cloning into 'py-motmetrics'...
remote: Enumerating objects: 1689, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 1689 (delta 29), reused 109 (delta 21), pack-reused 1564
Receiving objects: 100% (1689/1689), 6.95 MiB | 7.77 MiB/s, done.
Resolving deltas: 100% (1125/1125), done.
/content/py-motmetrics


In [ ]:
! pip install xmltodict

In [ ]:
import motmetrics as mm
# List all default metrics
mh = mm.metrics.create()
print(mh.list_metrics_markdown())

In [ ]:
%mkdir motmetrics/data/CenterTrack_Vehicles_TestEval/gt

In [ ]:
%mv motmetrics/data/CenterTrack_Vehicles_TestEval/gt.txt motmetrics/data/CenterTrack_Vehicles_TestEval/gt

In [ ]:
%mv motmetrics/data/CenterTrack_Vehicles_TestEval/test.txt motmetrics/data/CenterTrack_Vehicles_TestEval/CenterTrack_Vehicles_TestEval.txt

In [ ]:
! python -m motmetrics.apps.evaluateTracking motmetrics/data/ motmetrics/data/CenterTrack_Vehicles_TestEval/

usage: evaluateTracking.py [-h] [--log LOG] [--loglevel LOGLEVEL] [--fmt FMT]
                           [--solver SOLVER] [--skip SKIP] [--iou IOU]
                           groundtruths tests seqmap
evaluateTracking.py: error: the following arguments are required: seqmap


In [ ]:
! python -m motmetrics.apps.evaluateTracking

usage: evaluateTracking.py [-h] [--log LOG] [--loglevel LOGLEVEL] [--fmt FMT]
                           [--solver SOLVER] [--skip SKIP] [--iou IOU]
                           groundtruths tests seqmap
evaluateTracking.py: error: the following arguments are required: groundtruths, tests, seqmap


In [ ]:
! pip install bdd100k

#Convert MOT result to COCO

In [ ]:
import json
import numpy as np
import os
from collections import defaultdict
split = 'val_half'

DET_PATH = '../../data/mot17/'
ANN_PATH = '../../data/mot17/annotations/{}.json'.format(split)
OUT_DIR = '../../data/mot17/results/'
OUT_PATH = OUT_DIR + '{}_det.json'.format(split)

if __name__ == '__main__':
  if not os.path.exists(OUT_DIR):
    os.mkdir(OUT_DIR)
  seqs = [s for s in os.listdir(DET_PATH) if '_det' in s]
  data = json.load(open(ANN_PATH, 'r'))
  images = data['images']
  image_to_anns = defaultdict(list)
  for seq in sorted(seqs):
    print('seq', seq)
    seq_path = '{}/{}/'.format(DET_PATH, seq)
    if split == 'val_half':
      ann_path = seq_path + 'det/det_val_half.txt'
      train_ann_path = seq_path + 'det/det_train_half.txt'
      train_anns = np.loadtxt(train_ann_path, dtype=np.float32, delimiter=',')
      frame_base = int(train_anns[:, 0].max())
    else:
      ann_path = seq_path + 'det/det.txt'
      frame_base = 0
    if not IS_THIRD_PARTY:
      anns = np.loadtxt(ann_path, dtype=np.float32, delimiter=',')
    for i in range(len(anns)):
      frame_id = int(anns[i][0])
      file_name = '{}/img1/{:06d}.jpg'.format(seq, frame_id + frame_base)
      bbox = (anns[i][2:6]).tolist()
      score = 1 # float(anns[i][8])
      image_to_anns[file_name].append(bbox + [score])

  results = {}
  for image_info in images:
    image_id = image_info['id']
    file_name = image_info['file_name']
    dets = image_to_anns[file_name]
    results[image_id] = []
    for det in dets:
      bbox = [float(det[0]), float(det[1]), \
              float(det[0] + det[2]), float(det[1] + det[3])]
      ct = [(bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2]
      results[image_id].append(
        {'bbox': bbox, 'score': float(det[4]), 'class': 1, 'ct': ct})
  out_path = OUT_PATH
  json.dump(results, open(out_path, 'w'))

# DEMO our our model


In [ ]:
! pip install --upgrade pip
! pip install opencv-contrib-python

     |████████████████████████████████| 1.6MB 36.3MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [ ]:
%%time
! python demo.py tracking --load_model /content/CenterTrack/models/model_last.pth --demo /content/dinhtu/MyDrive/vehicles_detection/Tu_workspace/video_demo_tracking.mp4 --video_h 512 --video_w 512 --save_video

In [ ]:
%cd /content/CenterTrack
!pip install -U torch==1.4 torchvision==0.5 -f https://download.pytorch.org/whl/cu101/torch_stable.html
!pip install -U pytest==3.8 motmetrics==1.1.3 folium==0.2.1
!pip install -q -r requirements.txt

%cd /content/CenterTrack/src/lib/model/networks
!rm -rf DCNv2
!git clone https://github.com/CharlesShang/DCNv2.git

%cd /content/CenterTrack/src/lib/model/networks/DCNv2
!chmod u+x make.sh
!./make.sh

%cd /content/CenterTrack/src
import sys
from os.path import join
sys.path.insert(0, join('/content/CenterTrack', 'src/lib'))
sys.path.append(join('/content/CenterTrack', 'src'))
sys.path.append(join('/content/CenterTrack', 'src/lib/model/networks/DCNv2'))

In [ ]:
! pip install --upgrade six

ERROR: tensorflow 2.5.0 has requirement six~=1.15.0, but you'll have six 1.16.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement six~=1.15.0, but you'll have six 1.16.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0


In [ ]:
! pip install --upgrade torch torchvision

     |████████████████████████████████| 831.4MB 21kB/s 
     |████████████████████████████████| 22.1MB 131kB/s 
  Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0
  Found existing installation: torchvision 0.5.0
    Uninstalling torchvision-0.5.0:
      Successfully uninstalled torchvision-0.5.0


In [ ]:
#import sys
#CENTERTRACK_PATH = '/content/CenterTrack/src/lib'
#sys.path.insert(0, CENTERTRACK_PATH)
%cd /content/CenterTrack/src/lib
        
from detector import Detector
from opts import opts

MODEL_PATH = '/content/dinhtu/MyDrive/vehicles_detection/Tu_workspace/model_last.pth'
#MODEL_PATH = '/content/CenterTrack/src/lib/models/coco_tracking.pth'
TASK = 'tracking'
opt = opts().init('{} --load_model {}'.format(TASK, MODEL_PATH).split(' '))
detector = Detector(opt)

/content/CenterTrack/src/lib
Running tracking
Using tracking threshold for out threshold! 0.3
Fix size testing.
training chunk_sizes: [32]
input h w: 512 512
heads {'hm': 80, 'reg': 2, 'wh': 2, 'tracking': 2}
weights {'hm': 1, 'reg': 1, 'wh': 0.1, 'tracking': 1}
head conv {'hm': [256], 'reg': [256], 'wh': [256], 'tracking': [256]}
Creating model...
Using node type: (<class 'model.networks.dla.DeformConv'>, <class 'model.networks.dla.DeformConv'>)
loaded /content/CenterTrack/src/lib/models/coco_tracking.pth, epoch 70
Drop parameter base.pre_hm_layer.0.weight.
Drop parameter base.pre_hm_layer.1.weight.
Drop parameter base.pre_hm_layer.1.bias.
Drop parameter base.pre_hm_layer.1.running_mean.
Drop parameter base.pre_hm_layer.1.running_var.
Drop parameter base.pre_hm_layer.1.num_batches_tracked.


In [ ]:
#images = ['''image read from open cv or from a video''']
for img in images:
  ret = detector.run(img)['results']

In [ ]:
import cv2

file_path = '/content/dinhtu/MyDrive/vehicles_detection/Tu_workspace/video_demo_tracking.mp4'
cap = cv2.VideoCapture(file_path) 

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) 
frame_rate = int(cap.get(cv2.CAP_PROP_FPS)) #FPS
fourcc = cv2.VideoWriter_fourcc('m','p','4','v') 

out = cv2.VideoWriter('/content/dinhtu/MyDrive/vehicles_detection/Tu_workspace/video_demo_tracking/demo_CENTERTRACK.mp4', fourcc, frame_rate, (width, height))

while True:
  gg, img = cap.read() 
  if gg: 
    ret = detector.run(img)['results'] 
    for rret in ret:
       if rret.get('class') == 4: 
         cv2.rectangle(img, (int(rret.get('bbox')[0]), int(rret.get('bbox')[1])), (int(rret.get('bbox')[2]), int(rret.get('bbox')[3])), (0, 256, 0, 256), 2) 
         cv2.putText(img, str(rret.get('tracking_id')), (int(rret.get('bbox')[0])+30, int(rret.get('bbox')[1])+60), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 256), thickness=2)
  
       if rret.get('class') == 3: 
         cv2.rectangle(img, (int(rret.get('bbox')[0]), int(rret.get('bbox')[1])), (int(rret.get('bbox')[2]), int(rret.get('bbox')[3])), (0, 256, 0, 256), 2) 
         cv2.putText(img, str(rret.get('tracking_id')), (int(rret.get('bbox')[0])+30, int(rret.get('bbox')[1])+60), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 256), thickness=2)

       if rret.get('class') == 6: 
         cv2.rectangle(img, (int(rret.get('bbox')[0]), int(rret.get('bbox')[1])), (int(rret.get('bbox')[2]), int(rret.get('bbox')[3])), (0, 256, 0, 256), 2) 
         cv2.putText(img, str(rret.get('tracking_id')), (int(rret.get('bbox')[0])+30, int(rret.get('bbox')[1])+60), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 256), thickness=2)
       if rret.get('class') == 8: 
         cv2.rectangle(img, (int(rret.get('bbox')[0]), int(rret.get('bbox')[1])), (int(rret.get('bbox')[2]), int(rret.get('bbox')[3])), (0, 256, 0, 256), 2) 
         cv2.putText(img, str(rret.get('tracking_id')), (int(rret.get('bbox')[0])+30, int(rret.get('bbox')[1])+60), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 256), thickness=2)
    out.write(img)  
  else: 
    break
out.release()

In [ ]:
! python -m bdd100k.label.from_coco --annFile /content/dinhtu/MyDrive/vehicles_detection/Tu_workspace/test_yolo.json --save_path /content/dinhtu/MyDrive/kq_convert_tracking

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
